In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/omkarsadekar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/omkarsadekar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df = pd.read_csv('Reviews.csv')

In [3]:
df = df.loc[(df['Score'] > 2) & (df['Id'] < 15000)]

In [4]:
# Cleaning the text data
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/omkarsadekar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Preprocessing the text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Tokenize the text into individual words
    words = nltk.word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # Remove non-alphabetic characters
    words = [word for word in words if word.isalpha()]
    # Lemmatize words
    lemmatizer = nltk.stem.WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # Join the words back into a single string
    text = ' '.join(words)
    return text

In [6]:
# Applying preprocessing to the review text
df['clean_text'] = df['Text'].apply(preprocess_text)

In [7]:
# Combine all the reviews into a single string
all_text = ' '.join(df['clean_text'].tolist())

In [14]:
# Perform topic modeling using LDA
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words='english')
data_vectorized = vectorizer.fit_transform(df['clean_text'])

In [15]:
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(data_vectorized)

LatentDirichletAllocation(n_components=5, random_state=42)

In [16]:
# Display the top words for each topic
words = vectorizer.get_feature_names()
topic_keywords = []

for topic_weights in lda_model.components_:
    top_keywords = [words[index] for index in topic_weights.argsort()[:-6:-1]]
    topic_keywords.append(top_keywords)

topic_keywords_df = pd.DataFrame(topic_keywords)
topic_keywords_df.columns = ['Word 1', 'Word 2', 'Word 3', 'Word 4', 'Word 5']
topic_keywords_df.index = ['Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5']

/Users/omkarsadekar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
# Display the topic keywords using bamboolib

topic_keywords_df = topic_keywords_df.reset_index()
topic_keywords_df